In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import datetime as dt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
import seaborn as sns
import statistics
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
import datetime as dt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
#importing MAC season 4 - days to flowering trait file
flowering=pd.read_csv("/Users/ishitadebnath/Desktop/Genomics/MAC season 4/Phenotypes/Modified/mac_season_4_days_gdd_to_flowering.csv" , delimiter=",")
flowering=flowering.rename(columns={"genotype": "cultivar"})

In [3]:
#Importing and merging the gene and trait dataset
dist=pd.read_csv("/Users/ishitadebnath/Desktop/Genomics/Distance Matrix/allcultivars.csv")

s=dist['Column1'].tolist()
dist.columns = dist.columns[:0].tolist() + s
dist=dist.drop([0])
dist=dist.rename(columns={np.nan: "cultivar"})
dist=dist.fillna(0)

flowering=flowering.sort_values("cultivar",axis=0, ascending=True)
flowering=pd.merge(flowering, dist, on="cultivar", how='inner')

In [4]:
#Creating a feature space and standardizing the features. 

d=flowering.drop(columns=['cultivar','days_to_flowering',  'treatment', 'blocking_height',
       'scientificname','method_type', 'date_of_flowering', 'gdd_to_flowering', 'method','plot','range','column'])

X=d.to_numpy()
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = flowering[['days_to_flowering']]
y = np.asarray(y).flatten()

In [5]:
# Hyper parameter tuning

In [6]:
model = xgb.XGBRegressor()
groups=flowering.cultivar
n_estimators = range(10, 110, 10)
learning_rate = np.linspace(0,1,11)
max_depth = range(1, 20, 2)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[22:51:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best: -10.656271 using {'learning_rate': 0.4, 'max_depth': 3, 'n_estimators': 10}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 10}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 20}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 30}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 40}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 50}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 60}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 70}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 80}
-68.887724 (4.197385) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 90}
-68.887724 (4.19738

In [7]:
model = RandomForestRegressor()
max_depth = range(1, 20, 2)
n_estimators=range(100, 400, 10)
groups=flowering.cultivar
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.893749 using {'max_depth': 1, 'n_estimators': 100}
-10.893749 (1.260887) with: {'max_depth': 1, 'n_estimators': 100}
-10.943962 (1.280422) with: {'max_depth': 1, 'n_estimators': 110}
-10.998345 (1.281908) with: {'max_depth': 1, 'n_estimators': 120}
-10.981195 (1.271243) with: {'max_depth': 1, 'n_estimators': 130}
-11.029583 (1.277655) with: {'max_depth': 1, 'n_estimators': 140}
-10.966889 (1.302478) with: {'max_depth': 1, 'n_estimators': 150}
-10.984636 (1.330965) with: {'max_depth': 1, 'n_estimators': 160}
-11.006990 (1.390306) with: {'max_depth': 1, 'n_estimators': 170}
-11.032540 (1.366391) with: {'max_depth': 1, 'n_estimators': 180}
-10.966396 (1.289826) with: {'max_depth': 1, 'n_estimators': 190}
-10.959679 (1.328563) with: {'max_depth': 1, 'n_estimators': 200}
-10.944519 (1.301674) with: {'max_depth': 1, 'n_estimators': 210}
-11.031749 (1.309433) with: {'max_depth': 1, 'n_estimators': 220}
-11.053193 (1.277298) with: {'max_depth': 1, 'n_estimators': 230}
-10.914867 (1.2

In [8]:
model = SGDRegressor()
alpha = np.arange(0.01, 1.0, 0.05)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



Best: -10.932474 using {'alpha': 0.01}
-10.932474 (1.766220) with: {'alpha': 0.01}
-11.577755 (1.845767) with: {'alpha': 0.060000000000000005}
-11.489310 (1.962199) with: {'alpha': 0.11}
-11.094174 (1.785111) with: {'alpha': 0.16000000000000003}
-11.367823 (1.949500) with: {'alpha': 0.21000000000000002}
-11.613169 (1.644570) with: {'alpha': 0.26}
-11.539072 (1.968066) with: {'alpha': 0.31000000000000005}
-11.828209 (2.560726) with: {'alpha': 0.36000000000000004}
-12.053596 (2.053338) with: {'alpha': 0.41000000000000003}
-11.488415 (1.706215) with: {'alpha': 0.46}
-12.177577 (1.786026) with: {'alpha': 0.51}
-12.726036 (2.108401) with: {'alpha': 0.56}
-11.648955 (2.016638) with: {'alpha': 0.6100000000000001}
-12.365320 (1.990267) with: {'alpha': 0.66}
-12.852139 (2.207198) with: {'alpha': 0.7100000000000001}
-12.850330 (2.294648) with: {'alpha': 0.76}
-12.666886 (1.939202) with: {'alpha': 0.81}
-12.722879 (2.221072) with: {'alpha': 0.8600000000000001}
-12.488618 (2.102627) with: {'alpha'

In [9]:
model = LassoLars()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -11.337354 using {'alpha': 0.44999999999999984}
-22.229248 (18.167241) with: {'alpha': 0.1}
-21.734201 (17.083828) with: {'alpha': 0.11}
-21.265206 (16.307455) with: {'alpha': 0.12}
-20.864649 (15.679567) with: {'alpha': 0.13}
-20.531640 (15.166982) with: {'alpha': 0.13999999999999999}
-20.494501 (15.231558) with: {'alpha': 0.14999999999999997}
-20.788445 (15.932370) with: {'alpha': 0.15999999999999998}
-21.122986 (16.624942) with: {'alpha': 0.16999999999999998}
-21.415448 (17.283764) with: {'alpha': 0.17999999999999997}
-21.644109 (17.839991) with: {'alpha': 0.18999999999999995}
-21.798062 (18.250118) with: {'alpha': 0.19999999999999996}
-21.792270 (18.342156) with: {'alpha': 0.20999999999999996}
-21.766798 (18.393998) with: {'alpha': 0.21999999999999995}
-21.313821 (17.590235) with: {'alpha': 0.22999999999999995}
-20.863287 (16.786986) with: {'alpha': 0.23999999999999994}
-20.408940 (15.988343) with: {'alpha': 0.24999999999999992}
-19.950048 (15.194987) with: {'alpha': 0.259999

In [10]:
model = Lasso()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -10.850048 using {'alpha': 0.1}
-10.850048 (2.050009) with: {'alpha': 0.1}
-10.855017 (2.026827) with: {'alpha': 0.11}
-10.867328 (2.008431) with: {'alpha': 0.12}
-10.888365 (1.988037) with: {'alpha': 0.13}
-10.909911 (1.977669) with: {'alpha': 0.13999999999999999}
-10.945783 (1.979353) with: {'alpha': 0.14999999999999997}
-10.991165 (1.971039) with: {'alpha': 0.15999999999999998}
-11.042432 (1.967901) with: {'alpha': 0.16999999999999998}
-11.092304 (1.970598) with: {'alpha': 0.17999999999999997}
-11.137093 (1.978165) with: {'alpha': 0.18999999999999995}
-11.182950 (1.985731) with: {'alpha': 0.19999999999999996}
-11.228950 (1.993551) with: {'alpha': 0.20999999999999996}
-11.266311 (2.008139) with: {'alpha': 0.21999999999999995}
-11.296591 (2.004889) with: {'alpha': 0.22999999999999995}
-11.315399 (1.998048) with: {'alpha': 0.23999999999999994}
-11.320457 (1.983196) with: {'alpha': 0.24999999999999992}
-11.323035 (1.970720) with: {'alpha': 0.2599999999999999}
-11.325856 (1.958388)

In [11]:
model = Ridge()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -11.113469 using {'alpha': 0.9899999999999995}
-11.548984 (2.287507) with: {'alpha': 0.1}
-11.536259 (2.279498) with: {'alpha': 0.11}
-11.524100 (2.271710) with: {'alpha': 0.12}
-11.512458 (2.264139) with: {'alpha': 0.13}
-11.501289 (2.256779) with: {'alpha': 0.13999999999999999}
-11.490555 (2.249625) with: {'alpha': 0.14999999999999997}
-11.480224 (2.242672) with: {'alpha': 0.15999999999999998}
-11.470266 (2.235914) with: {'alpha': 0.16999999999999998}
-11.460655 (2.229343) with: {'alpha': 0.17999999999999997}
-11.451368 (2.222956) with: {'alpha': 0.18999999999999995}
-11.442384 (2.216744) with: {'alpha': 0.19999999999999996}
-11.433684 (2.210703) with: {'alpha': 0.20999999999999996}
-11.425252 (2.204827) with: {'alpha': 0.21999999999999995}
-11.417073 (2.199109) with: {'alpha': 0.22999999999999995}
-11.409131 (2.193544) with: {'alpha': 0.23999999999999994}
-11.401415 (2.188127) with: {'alpha': 0.24999999999999992}
-11.393914 (2.182852) with: {'alpha': 0.2599999999999999}
-11.38

In [12]:
model = ElasticNet()
alpha = np.arange(0.01, 1.0, 0.05)
l1_ratio=np.arange(0, 1, 0.01)
groups=flowering.cultivar
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 42.58699234218875, tolerance: 1.6058671755725191
  positive)


Best: -10.751900 using {'alpha': 0.01, 'l1_ratio': 0.99}
-11.106381 (2.012147) with: {'alpha': 0.01, 'l1_ratio': 0.0}
-11.106426 (2.013407) with: {'alpha': 0.01, 'l1_ratio': 0.01}
-11.106562 (2.014680) with: {'alpha': 0.01, 'l1_ratio': 0.02}
-11.106817 (2.015966) with: {'alpha': 0.01, 'l1_ratio': 0.03}
-11.107178 (2.017353) with: {'alpha': 0.01, 'l1_ratio': 0.04}
-11.107614 (2.018767) with: {'alpha': 0.01, 'l1_ratio': 0.05}
-11.108156 (2.020172) with: {'alpha': 0.01, 'l1_ratio': 0.06}
-11.108776 (2.021631) with: {'alpha': 0.01, 'l1_ratio': 0.07}
-11.109439 (2.023035) with: {'alpha': 0.01, 'l1_ratio': 0.08}
-11.110181 (2.024411) with: {'alpha': 0.01, 'l1_ratio': 0.09}
-11.110879 (2.025756) with: {'alpha': 0.01, 'l1_ratio': 0.1}
-11.111584 (2.027113) with: {'alpha': 0.01, 'l1_ratio': 0.11}
-11.112430 (2.028433) with: {'alpha': 0.01, 'l1_ratio': 0.12}
-11.113331 (2.029726) with: {'alpha': 0.01, 'l1_ratio': 0.13}
-11.114205 (2.031001) with: {'alpha': 0.01, 'l1_ratio': 0.14}
-11.115116 (2.0

In [13]:
#Implementation of different algorithms

# 5 fold cross-validation for Random Forest
groups=flowering.cultivar
ns=5
cv = GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X, y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('TRAIN :',X_train.shape, 'TEST :', X_test.shape)
    crf=RandomForestRegressor(max_depth=1, n_estimators=310).fit(X_train, y_train)

    pred=crf.predict(X_test)
    error=error+math.sqrt(np.sum((pred-y_test)**2)/y_test.shape[0])
    R2=R2+r2_score(y_test, pred)
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    error=error+math.sqrt(mean_squared_error(y_test, pred))

std= (statistics.stdev(e))
print('R^2 for 5-fold CV RF:', R2/ns)
print('Root Mean square error for 5-fold CV RF',error/ns,'+-', std)

# 5 fold cross-validation for Stochastic Gradient Descent Regression

cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=SGDRegressor(alpha=0.01).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV SG:', R2/ns)
print('Root Mean square error for 5-fold CV SG:',error/ns, '+-', std)

# 5 fold cross-validation for Linear Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LinearRegression().fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV LR:', R2/ns)
print('Root Mean square error for 5-fold CV LR:',error/ns, '+-', std)

# 5 fold cross-validation for XgBoost Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=xgb.XGBRegressor(booster='gbtree',importance_type='gain',learning_rate= 0.7, max_depth= 3, n_estimators= 150).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV XG:', R2/ns)
print('Root Mean square error for 5-fold CV XG:',error/ns, '+-', std)

# 5 fold cross-validation for LassoLars Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LassoLars(alpha=0.45).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV LL:', R2/ns)
print('Root Mean square error for 5-fold CV LL:',error/ns, '+-', std)

# 5 fold cross-validation for Lasso Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Lasso(alpha=0.1).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV L:', R2/ns)
print('Root Mean square error for 5-fold CV L:',error/ns, '+-', std)

# 5 fold cross-validation for Ridge Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Ridge(alpha=0.99, max_iter=5000).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV R:', R2/ns)
print('Root Mean square error for 5-fold CV R:',error/ns, '+-', std)

# 5 fold cross-validation for ElasticNet Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=ElasticNet(alpha= 0.01, l1_ratio= 0.99, max_iter=5000).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV EN:', R2/ns)
print('Root Mean square error for 5-fold CV EN:',error/ns, '+-', std)



TRAIN : (104, 364) TEST : (27, 364)
TRAIN : (105, 364) TEST : (26, 364)
TRAIN : (105, 364) TEST : (26, 364)
TRAIN : (105, 364) TEST : (26, 364)
TRAIN : (105, 364) TEST : (26, 364)
R^2 for 5-fold CV RF: -0.2535284353359525
Root Mean square error for 5-fold CV RF 21.986125044065634 +- 1.4692673493037094
R^2 for 5-fold CV SG: -0.20547287539346767
Root Mean square error for 5-fold CV SG: 10.951790163535062 +- 2.041296864056239
R^2 for 5-fold CV LR: -9.146337733408486e+19
Root Mean square error for 5-fold CV LR: 92766333717.25035 +- 51452384198.2415
[23:18:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:18:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:18:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:18:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2588375162588363, tolerance: 1.2585961904761904
  positive)
